In [1]:
# Search
import os
from dotenv import load_dotenv
load_dotenv(override=True)

os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY", None)
os.environ['LANGCHAIN_PROJECT']="agentic_rag_bam"

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY", None)

In [2]:
## LLM

import warnings
warnings.filterwarnings("ignore")

# from dotenv import load_dotenv
# load_dotenv(override=True)

from genai import Client, Credentials
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    TextGenerationParameters,
)
from langchain_ibm import WatsonxLLM, ChatWatsonx
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams


def bam_model(model_id='meta-llama/llama-3-8b-instruct', decoding_method='greedy', max_new_tokens=1000, 
              min_new_tokens=1, temperature=0.1, top_k=50, top_p=1, repetition_penalty=1):

    if decoding_method == 'greedy':
        decoding_method = DecodingMethod.GREEDY
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            repetition_penalty=repetition_penalty
        )
    else:
        decoding_method = DecodingMethod.SAMPLE
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty
        )

    llm = LangChainInterface(
        model_id=model_id,
        client=Client(credentials=Credentials.from_env()),
        parameters=parameters,
    )

    return llm

def watsonx_model(model_id="mistralai/mixtral-8x7b-instruct-v01", decoding_method='greedy', max_new_tokens=1000, 
                  min_new_tokens=1, temperature=0.5, top_k=50, top_p=1, repetition_penalty=1):
    params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.TOP_K: top_k,
        GenParams.TOP_P: top_p,
        GenParams.REPETITION_PENALTY: repetition_penalty
    }
    ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
    project_id = os.getenv("PROJECT_ID", None)
    api_key = os.getenv("API_KEY")
    watsonx_llm = ChatWatsonx(
        model_id=model_id,
        url=ibm_cloud_url,
        apikey=api_key,
        project_id=project_id,
        params=params,
    )
    return watsonx_llm
# prompt = "Tell me about IBM."
# print(f"Prompt: {prompt}")

# local_llm = bam_model()
local_llm = watsonx_model()
# print(local_llm.invoke(prompt))


# local_llm = "llama3"
# model_tested = "meta-llama/llama-3-8b-instruct"
# metadata = f"Agentic-RAG, {model_tested}"

In [3]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
# from langchain.python import PythonREPL
# from langchain.chat_models import ChatOpenAI

In [4]:
tools = load_tools(["llm-math","wikipedia"], llm=local_llm)

In [5]:
agent = initialize_agent(
    tools, 
    local_llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True
)

/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [6]:
agent("What is the 25% of 300?")

/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Thought: I need to calculate 25% of 300. I can do this by multiplying 300 by 0.25, which is the decimal equivalent of 25%. I will use the calculator tool for this.

Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 0.25"
}
```
Observation:
Observation: Answer: 75.0
Thought:

Final Answer: 75.0 is 25% of 300.

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0 is 25% of 300.'}

### Wikipedia example

In [7]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...

Thought: I need to find out who Tom M. Mitchell is and what books he has written. I will use Wikipedia to find this information.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```
Observation:
Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Art

In [8]:
print(result)

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?', 'output': 'Tom M. Mitchell wrote the book "Machine Learning".'}


### Python Agent

In [9]:
agent = create_python_agent(
    local_llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [10]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [11]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



To answer this question, I need to sort the given list of lists by the first element (last name) and then by the second element (first name). I can do this using the built-in `sort()` function in Python, which sorts a list in-place.

Action: Python_REPL
Action Input: 
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
customers.sort(key=lambda x: (x[0], x[1]))
print(customers)

Observation:
Observation: SyntaxError('invalid syntax', ('<string>', 5, 13, 'Observation:\n', 5, 13))
Thought:
It seems I made a mistake in my previous action. I should have printed the customers list after sorting it. Here is the corrected action.

Action: Python_REPL
Action Input: 
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
customers.sort(key=lambda x: (x[0], x[1]))
print(customers)

Observation:
Observat

"customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]\ncustomers.sort(key=lambda x: (x[0], x[1]))\nprint(customers)\n\nThis will output the sorted list: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]"

In [12]:
import langchain
langchain.debug=True

agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatWatsonx] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get a

### Define your own tool

In [13]:
from langchain.agents import tool
from datetime import date

In [14]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [15]:
agent = initialize_agent(
    tools + [time], 
    local_llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [16]:
result = agent("whats the date today?") 




> Entering new AgentExecutor chain...

Thought: The user is asking for the current date. I can use the 'time' tool to find this out.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```
Observation:
Observation: 2024-08-07
Thought:

Final Answer: The date today is 2024-08-07.

> Finished chain.


### multi-index agentic experiment

In [33]:
# import logging
from elasticsearch import Elasticsearch
from typing import List


class ContextRetriever:
    def __init__(self, es_credentials, model_id=".elser_model_2"):
        self.es_credentials = es_credentials
        # print(self.es_credentials)
        self.client = Elasticsearch(
            self.es_credentials["url"],
            basic_auth=(
                self.es_credentials["username"],
                self.es_credentials["password"],
            ),
            verify_certs=False,
            request_timeout=3600,
        )
        self.embedding_model_id = model_id

        # logging.info("get all indexes for this es client: {}".format(self.client.indices.get_alias(index="*")))

    def get_relevant_docs_elser(self, es_index_name, text_query="", top_k=3):
        # This function assumes the use of an ElasticSearch client setup earlier in the code
        # using .elser_model_2 as default
        elastic_query = {
            "query": {
                "bool": {
                    "must": [
                        {
                            "text_expansion": {
                                "ml.tokens": {
                                    "model_id": self.embedding_model_id,
                                    "model_text": text_query,
                                }
                            }
                        }
                    ]
                }
            },
            "_source": {"excludes": ["ml.tokens"]},
        }

        response = self.client.search(
            index=es_index_name, size=top_k, body=elastic_query
        )
        # return response
        clean_response = self.clean_elastic_output(response)
        return clean_response

    def clean_elastic_output(self, elastic_response):
        hits = elastic_response["hits"]["hits"]
        if len(hits) == 0:
            # logging.debug("No match found!")
            return []

        seen_paragraphs, cleaned_response = set(), list()
        for hit in hits:
            # print(hit["_source"].keys())  # To see what keys are available at this level
            score = hit["_score"]
            index_name = hit["_index"]
            doc_id = hit["_id"]
            text = hit["_source"]["text"]
            try:
                source = hit["_source"]["metadata"]["source"]
            except Exception as e:
                source = ""
            try:
                curated_answer = hit["_source"]["metadata"]["answer"]
            except Exception as e:
                curated_answer = ""
            try:
                try:
                    template_id = hit["_source"]["metadata"]["template_id"]
                except Exception as e:
                    template_id = hit["_source"]["metadata"]["templateID"]
            except Exception as e:
                template_id = ""
               
            # # Metadata dictionary with extracted information
            metadata = {
                "doc_id": doc_id,
                "index": index_name,
                "source": source,
                "text": text,
                "curated_answer": curated_answer,
                "template_id": template_id,
                "score": score,
            }
            if curated_answer not in seen_paragraphs:
                seen_paragraphs.add(curated_answer)
                cleaned_response.append(metadata)

        return cleaned_response
    
    def get_context(self, query, es_index_names: List, num_chunks=3):
        documents = list()
        for es_index in es_index_names:
            # logging.info(
            #     f"fetching context from es_index: {es_index}, query :  {query} top_k : {num_chunks}"
            # )
            documents.extend(
                self.get_relevant_docs_elser(
                    es_index_name=es_index, text_query=query, top_k=num_chunks
                )
            )

        sorted_documents = sorted(documents, key=lambda x: x["score"], reverse=True)[:3]

        context = [ctx["text"] if ctx["curated_answer"] == "" else ctx["curated_answer"] for ctx in sorted_documents]
        sources = [ctx["source"] for ctx in sorted_documents if ctx["source"]]
        template_ids = [ctx["template_id"] for ctx in sorted_documents]

        return "Context: \n\n{} \n\nSource: {}".format("\n".join(context), list(set(sources)))


In [34]:
_ = load_dotenv(override=True)

es_credentials = {
    "url": os.getenv("ES_URL"),
    "username": os.getenv("ES_USERNAME"),
    "password": os.getenv("ES_PASSWORD"),
    "certificate": os.getenv(
        "ES_CERTIFICATE_PATH"
    ),  # config.get("ElasticSearch", "es_certificate_path"),
}
import ast
es_index_names = ast.literal_eval(os.getenv("ES_INDEX_NAMES"))

_retriever = ContextRetriever(es_credentials)

In [42]:
@tool
def es_retrieve_it(query: str) -> str:
    """Returns context from ElasticSearch index, use this to get context with source embeddedfor IT questions \
    related to Ontario Government. \
    and when using the output from this tool to answer the user query, make sure to embed the source information.\
    The input should always be an user query, and this function will always return the fetched context - \
    any other function call should occur outside this function"""
    return _retriever.get_context(query, [es_index_names['it_index'], es_index_names['others_index']])

@tool
def es_retrieve_hr(query: str) -> str:
    """Returns context from ElasticSearch index, use this to get context with source embedded for HR/Business questions \
    related to Ontario Government. \
    and when using the output from this tool to answer the user query, make sure to embed the source information.\
    The input should always be an user query, and this function will always return the fetched context - \
    any other function call should occur outside this function"""
    return _retriever.get_context(query, [es_index_names['hr_index'], es_index_names['others_index']])

In [43]:
agent = initialize_agent(
    tools + [es_retrieve_it, es_retrieve_hr], 
    local_llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [44]:
result = agent("As a co-op student, can I avail paid vacations?") 
result



> Entering new AgentExecutor chain...

Thought: To answer this question, I need to understand the HR policies of the Ontario Government regarding co-op students and paid vacations. I will use the es_retrieve_hr tool to fetch this information.

Action:
```
{
  "action": "es_retrieve_hr",
  "action_input": "Co-op student paid vacations in Ontario Government"
}
```
Observation:
Observation: Context: 

If you are a co-op student and would like to learn about your entitlements, please refer to your hiring package. Hiring packages can be found on the [Standard Information Packages page](https://intra.ontario.ca/ops/standard-information-packages?utm_source=chatbot&utm_medium=referral&utm_campaign=intranet-chatbot) on InsideOPS. 

Summer students are generally not eligible for paid vacation, but may request for unpaid time off at the discretion of their manager. Summer students are also not entitled to statutory holiday pay.
Co-op students cannot have their co-op contract extended, unless it

{'input': 'As a co-op student, can I avail paid vacations?',
 'output': 'The policy does not explicitly mention whether co-op students can avail paid vacations. It would be best to refer to your hiring package or contact your HR advisor to understand your specific entitlements as a co-op student.'}

In [45]:
result = agent("How do I configure MS teams?") 
result



> Entering new AgentExecutor chain...

Thought: To answer this question, I need to provide information on how to configure Microsoft Teams. I will use the wikipedia tool to get a general overview of the process and then provide additional details if necessary.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Microsoft Teams configuration"
}
```
Observation:
Observation: Page: Microsoft Configuration Manager
Summary: Microsoft Configuration Manager (ConfigMgr) is a systems management software product developed by Microsoft for managing large groups of computers providing remote control, patch management, software distribution, operating system deployment, and hardware and software  inventory management.
Configuration Manager supports the Microsoft Windows and Windows Embedded operating systems.  Previous versions also supported macOS (OS X), Linux or UNIX, as well as Windows Phone, Symbian, iOS and Android mobile operating systems.
As per the latest release cadence, starting

{'input': 'How do I configure MS teams?',
 'output': "To access the online web version portal of Microsoft Teams, simply go to www.office.com. Select the Teams icon to access the Microsoft Teams web application. The MS Teams classic app for desktop can be installed from Microsoft's website using this link: [Download MS Teams app](https://go.microsoft.com/fwlink/?linkid=2196106&clcid=0x1009&culture=en-ca&country=ca). You can also easily install Teams, along with other Microsoft applications on your mobile device. To create a Teams site, give the team a name, add a short description, and set the team as private. Team owners have options for customizing and managing a team. Select 'More options' next to a team you own to see them. 'Manage team' is where you’ll find these options and more in-depth settings all in one place. Make sure to add a co-owner(s) to your Teams site so that its management can continue in case the original site owner is not available or leaves the organization."}

In [46]:
result = agent("Why is my paycheck lower than expected?") 
result



> Entering new AgentExecutor chain...

Thought: I need to check if there have been any recent changes in the person's payroll that could explain this decrease. I can use the es_retrieve_hr tool to retrieve this information.

Action:
```
{
  "action": "es_retrieve_hr",
  "action_input": "Recent changes in employee's payroll"
}
```
Observation:
Observation: Context: 

The pay directives can be found on the [Pay Directives intranet page](https://intra.ontario.ca/tbs/cpslrc-pay-directives-sms?utm_source=chatbot&utm_medium=referral&utm_campaign=intranet-chatbot).
If you are expecting a pay increase, please confirm with your manager that the request has been submitted to Payroll for processing. 

If you have confirmed the request was submitted but the change is not reflected in your pay, you may speak to a Customer Service Agent who can review your request further.

OSS Pay and Benefits Services operates and provides regular biweekly pay to all clients by prioritizing the delivery of **Tim

{'input': 'Why is my paycheck lower than expected?',
 'output': 'I would recommend checking if your pay increase has been properly requested and processed. If it has and the change is not reflected in your pay, consider speaking to a Customer Service Agent for further review.'}

In [47]:
result = agent("who is my WDHP advisor?") 
result



> Entering new AgentExecutor chain...

Thought: I need to find out who the WDHP advisor is. I can use the es_retrieve_hr tool to search for this information in the HR/Business ElasticSearch index.

Action:
```
{
  "action": "es_retrieve_hr",
  "action_input": "WDHP advisor"
}
```
Observation:
Observation: Context: 

WDHP complaints are assigned according to ministry portfolios listed at the bottom of the [WDHP Services page](https://intra.ontario.ca/ops/hr-workplace-discrimination-and-harassment-prevention-wdhp-services?utm_source=chatbot&utm_medium=referral&utm_campaign=intranet-chatbot#ministry-portfolio-teams). 

Once you have submitted a [WDHP Complaint Form](https://forms.mgcs.gov.on.ca/dataset/130-7540-3005?utm_source=chatbot&utm_medium=referral&utm_campaign=intranet-chatbot), it will be assigned to a WDHP Advisor and you will be contacted shortly. 

If you have already submitted a complaint, you may contact the Team Lead assigned to your ministry for further information.
Miche

{'input': 'who is my WDHP advisor?',
 'output': 'The WDHP advisors are Michelle (MAG) Lee, Miriam (MAG) Lee, Mona (MGCS) Lee, Nickisha (MAG) Lee, Nicola (MCSCS) Lee, Nicolle (TBS) Lee, Nora (MAG) Lee, and Norman (MCCES Senior Environmental Officer Court and Client Representative Senior Policy Advisor Manager, Client Services & Support Team Lead).'}